# Transformacion y limpieza de los datos - Silver Layer

In [ ]:
#Parametros
simbolo='IBM'

#### Cargamos el csv del lakehouse

In [ ]:
df_raw = spark.read.option('header', 'true').option("inferSchema", "true").csv('Files/IBM_finance_data.csv')

#### Limpieza, filtrado y transformaciones basicas

In [ ]:
from pyspark.sql.functions import *
silver_df = df_raw.orderBy(col("timestamp").desc())
silver_df = silver_df.dropDuplicates(["timestamp"])

In [ ]:
silver_df = silver_df.filter(
        (col("open") > 0) & 
        (col("high") >= col("low")) &
        (col("high") >= col("open")) &
        (col("high") >= col("close")) &
        (col("low") <= col("open")) &
        (col("low") <= col("close")) &
        (col("volume") >= 0))

#### Agregamos metadatos (fecha de procesamiento)

In [ ]:
silver_df = silver_df.withColumn("fecha_procesado", current_date())

In [ ]:
display(silver_df)

In [ ]:
silver_df.write.mode('overwrite').format('delta').saveAsTable(f'{simbolo}_silver_data')